In [2]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Dense, RepeatVector
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from keras.callbacks import LearningRateScheduler
import keras.backend as K
import pandas as pd
import quandl
import numpy as np
from datetime import date
import math
from keras.callbacks import ReduceLROnPlateau


F:\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
df = pd.read_csv('data/0728 200user.csv')

In [30]:
df.head()

,p_norm,p_flex,ls,tariff,ls_move
0,296.10,303.90,1,0.288,0.0
1,274.66,325.34,1,0.288,1.0
2,311.65,288.35,1,0.288,1.0
3,314.81,285.19,1,0.288,1.0
4,304.68,0.00,1,0.288,1.0


In [8]:
#生成测试数据
test=df[['p_norm','tariff']]
scaler = MinMaxScaler(feature_range=(0, 1))
x_test=scaler.fit_transform(test[96*243-96*2:96*243-96*1])
real_test=df[['p_norm']][96*243-96*1:]

In [10]:
#生成up和down不同的96组信号

down = []
for i in range(96):
    row = []
    for j in range(96):
        if j < i:
            row.append(0)
        else:
            row.append(-1)
    down.append(row)
down = np.array(down)

up = []
for i in range(96):
    row = []
    for j in range(96):
        if j < i:
            row.append(0)
        else:
            row.append(1)
    up.append(row)
up = np.array(up)

In [19]:
#根据倒数第二天数据进行序列预测
#对信号归一化
scaled_down = np.where(down==0,0.5,0)
scaled_up = np.where(up==0,0.5,1)
test_up =[]
for i in range(96):
    a=np.hstack((np.array(scaled_up[i]).reshape(96,1),x_test))
    test_up.append(a)
test_up = np.array(test_up)

In [20]:
test_down =[]
for i in range(96):
    a=np.hstack((np.array(scaled_down[i]).reshape(96,1),x_test))
    test_down.append(a)
test_down = np.array(test_down)

In [21]:
model = load_model('model/0728 LSTM.h5')

In [26]:
pre_flex_up=[]
for i in range(96):
    scaler.fit_transform()
    a = test_up[i].reshape(1,96,3)
    pre_flex = model.predict(a)
    pre_flex_up.append(pre_flex)

In [28]:
pre_flex_up = np.array(pre_flex_up)

In [29]:
pre_flex_up

array([[[[ 5.4470785e-02],
         [ 2.1139491e-01],
         [ 3.5780254e-01],
         ...,
         [-7.3742645e-04],
         [-1.0572650e-03],
         [-1.7967202e-03]]],


       [[[-1.1092238e-03],
         [ 6.7792960e-02],
         [ 2.5206846e-01],
         ...,
         [-7.3754566e-04],
         [-1.0575332e-03],
         [-1.7968096e-03]]],


       [[[-1.1092238e-03],
         [ 1.2328714e-03],
         [ 8.2326010e-02],
         ...,
         [-7.3784369e-04],
         [-1.0575183e-03],
         [-1.7967053e-03]]],


       ...,


       [[[-1.1092238e-03],
         [ 1.2328714e-03],
         [ 5.3268520e-04],
         ...,
         [ 8.5533965e-01],
         [ 1.0749360e+00],
         [ 1.0069780e+00]]],


       [[[-1.1092238e-03],
         [ 1.2328714e-03],
         [ 5.3268520e-04],
         ...,
         [ 3.5686370e-02],
         [ 8.6041433e-01],
         [ 1.0804901e+00]]],


       [[[-1.1092238e-03],
         [ 1.2328714e-03],
         [ 5.3268520e-04],
     